<a href="https://colab.research.google.com/github/1muyassar-kholova/surkhandarya_map/blob/main/Areal_Surxondaryo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 1. Kerakli kutubxonalarni chaqiramiz
#import geopandas as gpd
import folium
from folium import Popup
from branca.element import Template, MacroElement
from google.colab import files


In [ ]:
# 2. Faylni yuklash (kompyuterdan)
uploaded = files.upload()


Saving gadm41_UZB_2.json to gadm41_UZB_2.json


In [ ]:
import geopandas as gpd
from shapely.geometry import shape
import json

# JSON faylni xom formatda o‘qiymiz
with open("/mnt/data/gadm41_UZB_2.json", "r", encoding="utf-8") as f:
    geojson_data = json.load(f)

# GeoDataFrame yaratamiz
features = [feature for feature in geojson_data["features"] if feature["properties"]["NAME_1"] == "Surxondaryo"]

gdf = gpd.GeoDataFrame.from_features(features)

# Faylni GeoJSON formatda saqlaymiz
output_path = "/mnt/data/surxondaryo.geojson"
gdf.to_file(output_path, driver="GeoJSON")

output_path


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/gadm41_UZB_2.json'

In [ ]:
# 2. Faylni yuklash (kompyuterdan)
uploaded = files.upload()


Saving surxondaryo.geojson to surxondaryo.geojson


In [ ]:
import geopandas as gpd

# GeoJSON faylni o'qish
gdf = gpd.read_file("surxondaryo.geojson")

# Ko'rib chiqamiz
gdf.head()


,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,UZB.12.1_1,UZB,Uzbekistan,UZB.12_1,Surxondaryo,NA,Angor,NA,NA,District,District,NA,UZ.SU.AN,"POLYGON ((67.1136 37.3145, 67.1327 37.3221, 67..."
1,UZB.12.2_1,UZB,Uzbekistan,UZB.12_1,Surxondaryo,NA,Bandixon,Bandikhon,NA,District,District,NA,UZ.SU.BA,"POLYGON ((67.4654 37.763, 67.4728 37.7736, 67...."
2,UZB.12.3_1,UZB,Uzbekistan,UZB.12_1,Surxondaryo,NA,Boysun,NA,NA,District,District,NA,UZ.SU.BO,"POLYGON ((66.6503 38.0314, 66.682 38.0237, 66...."
3,UZB.12.4_1,UZB,Uzbekistan,UZB.12_1,Surxondaryo,NA,Denov,Denau,NA,City,City,NA,UZ.SU.DC,"POLYGON ((67.8508 38.1604, 67.8743 38.1463, 67..."
4,UZB.12.5_1,UZB,Uzbekistan,UZB.12_1,Surxondaryo,NA,Jarqo'rg'on,Jarkurghon,NA,District,District,NA,UZ.SU.JA,"POLYGON ((67.8322 37.3599, 67.7994 37.3575, 67..."


In [ ]:
data = {
    "Qumqo'rg'on": {'ječki': 72, 'ečki': 49, 'rang': 'yellow'},
    "Jarqo'rg'on": {'ječki': 39, 'ečki': 49, 'rang': 'skyblue'},
    "Angor": {'ječki': 32, 'ečki': 47, 'rang': 'skyblue'},
    "Bandixon": {'ječki': 32, 'ečki': 13, 'rang': 'yellow'},
    "Sherobod": {'ječki': 28, 'ečki': 46, 'rang': 'skyblue'},
    "Sho'rchi": {'ječki': 45, 'ečki': 47, 'rang': 'skyblue'},
    "Boysun": {'ječki': 21, 'ečki': 35, 'rang': 'skyblue'},
    "Oltinsoy": {'ječki': 37, 'ečki': 64, 'rang': 'skyblue'},
    "Qiziriq": {'ječki': 26, 'ečki': 34, 'rang': 'skyblue'},
    "Termiz": {'ječki': 8, 'ečki': 46, 'rang': 'skyblue'},
    "Muzrabot": {'ječki': 34, 'ečki': 54, 'rang': 'skyblue'},
    "Denov": {'ječki': 10, 'ečki': 46, 'rang': 'skyblue'},
    "Uzun": {'ječki': 31, 'ečki': 32, 'rang': 'skyblue'},
    "Termiz City": {'ječki': 1, 'ečki': 21, 'rang': 'skyblue'},
    "Sariosiyo": {'ječki': 12, 'ečki': 17, 'rang': 'skyblue'}
}


In [ ]:
import folium
from branca.element import Template, MacroElement

def get_rang(tuman):
    return data.get(tuman, {}).get("rang", "gray")

def get_tooltip(tuman):
    if tuman in data:
        return f"{tuman}<br>ječki(yechki): {data[tuman]['ječki']}<br>ečki(echki): {data[tuman]['ečki']}"
    return tuman

m = folium.Map(location=[37.23, 67.28], zoom_start=8, tiles='cartodbpositron')

for _, row in gdf.iterrows():
    tuman = row['NAME_2']
    geo_json = folium.GeoJson(
        data=row.__geo_interface__,
        style_function=lambda feature, tuman=tuman: {
            'fillColor': get_rang(tuman),
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        }
    )
    geo_json.add_child(folium.Popup(get_tooltip(tuman)))
    geo_json.add_to(m)

# Legend
legend_html = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed;
    bottom: 50px;
    right: 50px;
    width: 160px;
    height: 90px;
    background-color: white;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    padding: 10px;
    ">
    <b>Ranglar izohi</b><br>
    <i style="background:yellow;opacity:0.8;width:12px;height:12px;display:inline-block"></i> ječki(yechki)<br>
    <i style="background:skyblue;opacity:0.8;width:12px;height:12px;display:inline-block"></i> ečki(echki)<br>
</div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

m


AttributeError: 'Series' object has no attribute '__geo_interface__'

In [ ]:
AttributeError: 'Series' object has no attribute '__geo_interface__'


SyntaxError: invalid syntax (<ipython-input-8-1aa860363759>, line 1)

In [ ]:
for _, row in gdf.iterrows():
    tuman = row['NAME_2']
    geo_json = folium.GeoJson(
        data=row['geometry'].__geo_interface__,  # <-- TO‘G‘RI
        style_function=lambda feature, tuman=tuman: {
            'fillColor': get_rang(tuman),
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        }
    )
    geo_json.add_child(folium.Popup(get_tooltip(tuman)))
    geo_json.add_to(m)


In [ ]:
m


In [ ]:
from folium.features import GeoJsonTooltip


In [ ]:
geo_json = folium.GeoJson(
    data=row['geometry'].__geo_interface__,
    style_function=lambda feature, tuman=tuman: {
        'fillColor': get_rang(tuman),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    },
    tooltip=GeoJsonTooltip(fields=[], aliases=[], labels=False),  # bu qatorni hozircha bo‘sh qoldiramiz
)


In [ ]:
geo_json.add_child(folium.Tooltip(tuman))


In [ ]:
for _, row in gdf.iterrows():
    tuman = row['NAME_2']
    geo_json = folium.GeoJson(
        data=row['geometry'].__geo_interface__,
        style_function=lambda feature, tuman=tuman: {
            'fillColor': get_rang(tuman),
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        }
    )
    geo_json.add_child(folium.Tooltip(tuman))  # 🔵 bu — tuman nomi tooltip sifatida
    geo_json.add_child(folium.Popup(get_tooltip(tuman)))
    geo_json.add_to(m)


In [ ]:
m

In [ ]:
for _, row in gdf.iterrows():
    tuman = row['NAME_2']
    geo_json = folium.GeoJson(
        data=row['geometry'].__geo_interface__,
        style_function=lambda feature, tuman=tuman: {
            'fillColor': get_rang(tuman),
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        }
    )
    geo_json.add_child(folium.Tooltip(tuman))
    geo_json.add_child(folium.Popup(get_tooltip(tuman)))
    geo_json.add_to(m)

    # 🟢 Yangi qator — tuman markaziga nom yozish
    center = row['geometry'].centroid
    folium.Marker(
        location=[center.y, center.x],
        icon=folium.DivIcon(html=f"<div style='font-size: 10pt; color: black;'><b>{tuman}</b></div>")
    ).add_to(m)


In [ ]:
m

In [ ]:
import pandas as pd


# "buzāv" so'zining tumanlardagi statistikasi
data = {
    'Region': [
        "Qumqo'rg'on tumani", "Jarqo'rg'on tumani", "Angor tumani", "Bandixon tumani",
        "Sherobod tumani", "Sho'rchi tumani", "Oltinsoy tumani", "Qiziriq tumani",
        "Termiz tumani", "Muzrabot tumani", "Denov tumani", "Uzun tumani",
        "Termiz shahri", "Sariosiyo tumani", "Boysun tumani"
    ],
    'buzav_count': [109, 78, 60, 42, 61, 84, 83, 54, 33, 68, 41, 47, 10, 19, 25]
}
df = pd.DataFrame(data)

# 4. Birlashtirish
merged = gdf.merge(df, left_on="name", right_on="Region")

# 5. Rang funksiyasi: 50 dan yuqori kuchli rang, pastroq och rang
def get_color(value):
    return '#ff4500' if value > 50 else '#ffd9b3'

merged['color'] = merged['buzav_count'].apply(get_color)

# 6. Xarita
m = folium.Map(location=[37.5, 67.0], zoom_start=8)

folium.GeoJson(
    merged,
    style_function=lambda feature: {
        'fillColor': feature['properties']['color'],
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.75,
    },
    tooltip=GeoJsonTooltip(
        fields=['Region', 'buzav_count'],
        aliases=['Tuman', '“Buzāv” uchrash soni']
    )
).add_to(m)

# 7. Natijani ko‘rsatish
m

NameError: name 'gdf' is not defined

In [ ]:
import pandas as pd

# "buzāv" so'zining tumanlardagi statistikasi
data = {
    'Region': [
        "Qumqo'rg'on tumani", "Jarqo'rg'on tumani", "Angor tumani", "Bandixon tumani",
        "Sherobod tumani", "Sho'rchi tumani", "Oltinsoy tumani", "Qiziriq tumani",
        "Termiz tumani", "Muzrabot tumani", "Denov tumani", "Uzun tumani",
        "Termiz shahri", "Sariosiyo tumani", "Boysun tumani"
    ],
    'buzav_count': [109, 78, 60, 42, 61, 84, 83, 54, 33, 68, 41, 47, 10, 19, 25]
}

df = pd.DataFrame(data)


In [ ]:
import os
os.listdir()


['.config', 'sample_data']

In [ ]:
"surxondaryo_tumanlari.geojson"


'surxondaryo_tumanlari.geojson'

In [ ]:
gdf = gpd.read_file("surxondaryo_tumanlari.geojson")


NameError: name 'gpd' is not defined

In [ ]:
import geopandas as gpd


In [ ]:
gdf = gpd.read_file("surxondaryo_tumanlari.geojson")


DataSourceError: surxondaryo_tumanlari.geojson: No such file or directory

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving surxondaryo.geojson to surxondaryo.geojson


In [ ]:
import geopandas as gpd
gdf = gpd.read_file("surxondaryo_tumanlari.geojson")


DataSourceError: surxondaryo_tumanlari.geojson: No such file or directory

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving gadm41_UZB_2.json to gadm41_UZB_2.json


In [ ]:
import geopandas as gpd
gdf = gpd.read_file("gdf = gpd.read_file("gadm41_UZB_2.geojson")
geojson")


SyntaxError: unterminated string literal (detected at line 2) (<ipython-input-19-807c6e183882>, line 2)

In [ ]:
gdf = gpd.read_file("gdf = gpd.read_file("gadm41_UZB_2.geojson")


SyntaxError: unterminated string literal (detected at line 1) (<ipython-input-20-6a4239c2ef81>, line 1)

In [ ]:
gdf = gpd.read_file("gadm41_UZB_2.geojson")


DataSourceError: gadm41_UZB_2.geojson: No such file or directory

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving gadm41_UZB_2.json to gadm41_UZB_2 (1).json
